# Zenodo API code for getting GitHub Repos 

### Setup  

Ensure you are in the `coding-smart-github` conda environment and have the following python packages in your environment:
  * requests    

#### Github Authentication

Create an access token via Zenodo Applications settings [here](https://zenodo.org/account/settings/applications/tokens/) and create a file called `zenodoconfig.cfg` with the following content:

    [ACCESS]
    token = <your-access-token>
    
Ensure you've pasted in your token, but leave `[ACCESS]` and `token =` .

In [10]:
#import pandas as pd
#import numpy as np 
import os 
from os import path
import configparser 
import requests
import csv

In [8]:
# set up zenodo access token

config = configparser.ConfigParser()
config.read('../zenodoconfig.cfg')
config.sections()

access_token = config['ACCESS']['token']

r = requests.get('https://zenodo.org/api/deposit/depositions',
                  params={'access_token': access_token})

print(r)

r.status_code
# 200

#r.json()
# []


<Response [200]>


200

In [21]:
# get zenodo records with type software  
# via: https://github.com/npch/reporunner/blob/main/notebooks/zenodo.ipynb

search_query = "software"
r = requests.get("https://zenodo.org/api/records", params={'type': 'software'})

In [15]:
dir(r)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [18]:
r.json()

{'conceptdoi': '10.5281/zenodo.8435515',
 'conceptrecid': '8435515',
 'created': '2023-10-12T13:30:46.414361+00:00',
 'doi': '10.5281/zenodo.8435516',
 'files': [{'bucket': '5d139941-9d0f-46c4-84bd-e39f7a43897e',
   'checksum': 'md5:a86ba9d65527846fbbafd8783bae7a6b',
   'key': 'SushiLab/omrgc_v2_scripts-v1.0.0.zip',
   'links': {'self': 'https://zenodo.org/api/files/5d139941-9d0f-46c4-84bd-e39f7a43897e/SushiLab/omrgc_v2_scripts-v1.0.0.zip'},
   'size': 3123746,
   'type': 'zip'}],
 'id': 8435516,
 'links': {'badge': 'https://zenodo.org/badge/doi/10.5281/zenodo.8435516.svg',
  'bucket': 'https://zenodo.org/api/files/5d139941-9d0f-46c4-84bd-e39f7a43897e',
  'conceptbadge': 'https://zenodo.org/badge/doi/10.5281/zenodo.8435515.svg',
  'conceptdoi': 'https://doi.org/10.5281/zenodo.8435515',
  'doi': 'https://doi.org/10.5281/zenodo.8435516',
  'html': 'https://zenodo.org/record/8435516',
  'latest': 'https://zenodo.org/api/records/8435516',
  'latest_html': 'https://zenodo.org/record/8435516

In [22]:
identifiers = [] # create empty list for record IDs of SOFTWARE records   

if 'hits' in r.json():
    still_iterating = True
else:
    still_iterating = False

while still_iterating and (len(identifiers) < 50):
#while len(identifiers) < 50:
    if 'hits' in r.json():
        for hit in r.json()['hits']['hits']:
#            print(hit['id'])
#        print(hit['metadata']['title'])
            identifiers.append(hit['id'])
            
        if 'next' in r.json()['links']:
            r = requests.get(r.json()['links']['next'])
        else: 
            still_iterating = False
    else:
        still_iterating = False
        
print(identifiers)

[8436192, 8436186, 8436185, 8436176, 8436164, 8436160, 8436146, 8436118, 8436113, 8436090, 8436070, 8436065, 8436046, 8436041, 8422895, 8436030, 8436010, 8436003, 8435965, 8435958, 8435918, 8435829, 8435828, 8435736, 8435735, 8435731, 8435729, 8435720, 8435664, 8435646, 8435641, 8435614, 8435603, 8435554, 8435519, 8435516, 8435512, 8435495, 8435487, 8435486, 8435482, 8435478, 8435466, 8435463, 8435453, 8435438, 8435431, 8435345, 8435296, 8435277]


In [23]:
# Get GitHub links from a list of Zenodo IDs

f = open('results.csv', 'w')
writer = csv.writer(f)

header = ['Zenodo ID', 'GitHub Related Link', 'Relation Type']
writer.writerow(header)

for record_id in identifiers:
    r = requests.get("https://zenodo.org/api/records/{}".format(record_id))

    for id in r.json()['metadata']['related_identifiers']:
        if ("github.com" in id['identifier']):
            row = []
            row.append(record_id)
            row.append(id['identifier'])
            row.append(id['relation'])
            writer.writerow(row)

f.close()